In [1]:
import os
import json
import mne
import scipy

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
from scipy.signal import hilbert, coherence

try:
    import PyQt5.QtCore
    %matplotlib qt
except ImportError:
    %matplotlib inline

mne.set_log_level('WARNING')

In [2]:
import my_functions as my_fun
import my_visualization_functions as my_vis_fun

SyntaxError: '[' was never closed (my_visualization_functions.py, line 256)

In [ ]:
# Define subjects to process
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 20)]  
data_path = "eeg-motor-movementimagery-dataset-1.0.0/files/"

In [ ]:
# Process EEG data
eeg_data = my_fun.process_eeg(subjects=subjects, data_path=data_path, mode="automatic", apply_ica=False)

## Visualization for one subject

In [ ]:
subject = "S001" 
condition = "real_right_hand"  # real_left_hand or imagined_left_hand
epochs = eeg_data[subject][condition][0]

In [ ]:
my_vis_fun.plot_raw_eeg(eeg_data[subject][condition][0], subject, condition)

In [ ]:
my_vis_fun.plot_erd_ers(epochs, subject, condition)

## PLV and Coherence Pipeline for one subject

In [ ]:
real_epochs = mne.concatenate_epochs(eeg_data[subject]['real_right_hand'])
imagined_epochs = mne.concatenate_epochs(eeg_data[subject]['imagined_right_hand'])
rest_epochs = mne.concatenate_epochs(eeg_data[subject]['rest'])

In [ ]:
plv_real = my_fun.compute_plv_matrix(real_epochs)
plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
plv_rest = my_fun.compute_plv_matrix(rest_epochs)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real, real_epochs.ch_names, title="PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined, imagined_epochs.ch_names, title="PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_rest, rest_epochs.ch_names, title="PLV - Rest")

In [ ]:
my_vis_fun.plot_plv_difference(plv_real, plv_imagined, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real, plv_rest, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

In [ ]:
my_vis_fun.plot_motor_plv_difference(plv_real, plv_imagined, real_epochs.ch_names, title="PLV (Motor Cortex) - Real Right Hand")

## PLV and Coherence Pipeline for multiple subjects

In [ ]:
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 70)]  

plv_results = {"real": [], "imagined": [], "rest": []}

eeg_data_all = {}  # <-- Store all subjects

for subject in subjects:
    print(f"\n🚀 Processing {subject}...")
    
    eeg_data = my_fun.process_eeg([subject], data_path="eeg-motor-movementimagery-dataset-1.0.0/files/", mode="automatic", apply_ica=False)

    # Merge subject's data into master dict
    eeg_data_all.update(eeg_data)

    # Compute PLV matrix
    real_epochs = mne.concatenate_epochs(eeg_data[subject]["real_right_hand"])
    imagined_epochs = mne.concatenate_epochs(eeg_data[subject]["imagined_right_hand"])
    rest_epochs = mne.concatenate_epochs(eeg_data[subject]["rest"])

    plv_real = my_fun.compute_plv_matrix(real_epochs)
    plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
    plv_rest = my_fun.compute_plv_matrix(rest_epochs)

    plv_results["real"].append(plv_real)
    plv_results["imagined"].append(plv_imagined)
    plv_results["rest"].append(plv_rest)


In [ ]:
plv_real_group = np.mean(plv_results["real"], axis=0)
plv_imagined_group = np.mean(plv_results["imagined"], axis=0)
plv_rest_group = np.mean(plv_results["rest"], axis=0)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real_group, real_epochs.ch_names, title="Group Average PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Rest")

my_vis_fun.plot_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real_group, plv_rest_group, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

#my_vis_fun.plot_motor_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)

In [ ]:
# Define subjects and channel pairs
#subjects = ["S001", "S002", "S005"]
conditions = {"real": "real_right_hand", "imagined": "imagined_right_hand", "rest": "rest"} 

channel_pairs = [
    ("C3", "C4"),  # Primary motor cortex
    ("C1", "C2"),  # Central midline
    ("FC3", "FC4"),  # Frontal-motor
    ("Cz", "CPz"),  # Midline execution
    ("CP3", "CP4"),  # Parietal-motor
    ("Fz", "Cz"),  # Frontal-central connection
    ("O1", "O2"),  # Occipital 
    ("PO7", "PO8"),  # Parietal
    ("Fp1", "PO8")   # Prefrontal
]

# Analyze
df_plv_coh = my_fun.analyze_pairwise_plv_coherence(subjects, eeg_data_all, conditions, channel_pairs)

# Plot!
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="PLV Mean")
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="Coherence Mean")

In [ ]:
df_stats = my_fun.paired_ttest_plv(df_plv_coh, metric="PLV Mean")
my_fun.report_paired_ttests(df_stats)

# Time Frequency Representation Analysis

In [ ]:
# For each condition (real, imagined, rest)
freqs = np.arange(5, 40, 1)  # 5-40 Hz range
n_cycles = freqs / 2

power_real = my_fun.tfr_morlet(real_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)
power_imagined = my_fun.tfr_morlet(imagined_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)
power_rest = my_fun.tfr_morlet(rest_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)

In [ ]:
# Plot time-frequency plots for key channels (C3, C4)
power_real.plot_joint(title='Real Movement', picks=['C3', 'C4'])
power_imagined.plot_joint(title='Imagined Movement', picks=['C3', 'C4'])
power_rest.plot_joint(title='Rest', picks=['C3', 'C4'])

# Classification / Decoding Part

In [ ]:
# 1. Run classification on both condition pairs
group_results, individual_results = my_fun.classify_condition_pairs(eeg_data, subjects)

# 2. Visualize the comparison
summary = my_vis_fun.visualize_comparison(group_results, individual_results)

# Frequency Bands and Time Window Analysis

In [ ]:
# Run the frequency band analysis
band_results, band_names = my_fun.analyze_frequency_bands(eeg_data, subjects)

In [ ]:
# Run the time window analysis
window_results, window_names = my_fun.analyze_time_windows(eeg_data, subjects)

# New  Things 2

In [ ]:
import numpy as np

def compute_mean_topographies(epochs_dict, picks=None):
    """
    Compute the average topography for each condition from epochs.
    
    Parameters
    ----------
    epochs_dict : dict
        Dictionary with keys like 'rest', 'motor_execution', 'motor_imagery' and MNE Epochs as values.
    picks : list or None
        EEG channels to include.
    
    Returns
    -------
    topographies : dict
        Dictionary of condition -> average topography (n_channels,)
    """
    topographies = {}
    for condition, epochs in epochs_dict.items():
        data = epochs.get_data(picks=picks)  # shape: (n_epochs, n_channels, n_times)
        mean_topo = data.mean(axis=(0, 2))   # average over epochs and time
        topographies[condition] = mean_topo
    return topographies


In [ ]:
from scipy.spatial.distance import pdist, squareform

def compute_rdm(topographies):
    """
    Compute representational dissimilarity matrix (1 - Pearson correlation).
    
    Parameters
    ----------
    topographies : dict
        Dictionary of condition -> mean topography.
    
    Returns
    -------
    rdm : ndarray
        Square dissimilarity matrix (n_conditions x n_conditions).
    labels : list
        Condition names (for plotting).
    """
    labels = list(topographies.keys())
    vectors = np.array([topographies[label] for label in labels])
    
    # Dissimilarity = 1 - Pearson correlation
    correlation = np.corrcoef(vectors)
    dissimilarity = 1 - correlation
    
    return dissimilarity, labels


In [ ]:
from sklearn.manifold import MDS
import matplotlib.pyplot as plt

def plot_mds_from_rdm(rdm, labels, title="RSA: MDS Visualization"):
    """
    Project dissimilarity matrix into 2D using MDS and plot.
    """
    mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
    coords = mds.fit_transform(rdm)

    plt.figure(figsize=(6, 5))
    for i, label in enumerate(labels):
        plt.scatter(*coords[i], label=label, s=100)
        plt.text(coords[i, 0] + 0.01, coords[i, 1], label, fontsize=12)
    
    plt.title(title)
    plt.xlabel("MDS Dimension 1")
    plt.ylabel("MDS Dimension 2")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
from sklearn.manifold import TSNE

def plot_tsne_from_topographies(topographies, title="RSA: t-SNE Visualization"):
    """
    Use t-SNE to project topographies into 2D space.
    """
    labels = list(topographies.keys())
    vectors = np.array([topographies[label] for label in labels])
    
    tsne = TSNE(n_components=2, perplexity=5, random_state=42)
    coords = tsne.fit_transform(vectors)

    plt.figure(figsize=(6, 5))
    for i, label in enumerate(labels):
        plt.scatter(*coords[i], label=label, s=100)
        plt.text(coords[i, 0] + 0.01, coords[i, 1], label, fontsize=12)

    plt.title(title)
    plt.xlabel("t-SNE Dim 1")
    plt.ylabel("t-SNE Dim 2")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
epochs_dict = {
    'rest': combined_rest,
    'motor_execution': epochs_exec['task'],
    'motor_imagery': epochs_imag['task']
}


In [ ]:
topos = compute_mean_topographies(epochs_dict, picks=['C3', 'Cz', 'C4'])
rdm, labels = compute_rdm(topos)
plot_mds_from_rdm(rdm, labels)
# or
plot_tsne_from_topographies(topos)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.manifold import MDS

def compute_rsa_and_plot_mds(epochs_dict, picks=None, metric="correlation"):
    """
    Compute Representational Similarity (RSA) using correlation distance and visualize with MDS.
    
    Parameters:
    -----------
    epochs_dict : dict
        Dictionary with keys 'rest', 'motor_execution', 'motor_imagery' and values as mne.Epochs objects.
    picks : list or None
        Channel picks. If None, all channels are used.
    metric : str
        Distance metric to use (default = 'correlation').
    """
    condition_names = list(epochs_dict.keys())
    avg_patterns = []

    # Step 1: Compute mean spatial pattern for each condition
    for cond in condition_names:
        data = epochs_dict[cond].get_data(picks=picks)  # shape: (n_epochs, n_channels, n_times)
        avg_pattern = np.mean(data, axis=(0, 2))  # average over epochs and time -> shape: (n_channels,)
        avg_patterns.append(avg_pattern)

    avg_patterns = np.array(avg_patterns)

    # Step 2: Compute pairwise dissimilarities (e.g., 1 - correlation)
    dist_matrix = pairwise_distances(avg_patterns, metric=metric)

    # Step 3: MDS to project into 2D space
    mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
    coords = mds.fit_transform(dist_matrix)

    # Step 4: Plot
    plt.figure(figsize=(6, 5))
    for i, cond in enumerate(condition_names):
        plt.scatter(coords[i, 0], coords[i, 1], label=cond.upper(), s=100)
        plt.text(coords[i, 0], coords[i, 1], cond.upper(), fontsize=12, ha='center', va='center')

    plt.title("RSA (MDS Projection of Condition Similarities)")
    plt.xlabel("MDS Dimension 1")
    plt.ylabel("MDS Dimension 2")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    return dist_matrix, coords

# Assuming you want to run this with channels C3, Cz, and C4
compute_rsa_and_plot_mds(epochs_dict={"rest": combined_rest, "motor_execution": epochs_exec["task"], "motor_imagery": epochs_imag["task"]},
                         picks=["C3", "Cz", "C4"])


In [ ]:
def plot_tsne_all_trials(epochs_dict, picks=['C3', 'Cz', 'C4']):
    """
    Flatten and reduce all trials across conditions using t-SNE.
    """
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    X = []
    y = []

    for label, epochs in epochs_dict.items():
        data = epochs.get_data(picks=picks)  # shape: (n_epochs, n_channels, n_times)
        flat = data.reshape(data.shape[0], -1)  # flatten each epoch
        X.append(flat)
        y.extend([label] * len(flat))

    X = np.vstack(X)
    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    X_tsne = tsne.fit_transform(X)

    # Plot
    plt.figure(figsize=(7, 6))
    for label in np.unique(y):
        idx = [i for i, val in enumerate(y) if val == label]
        plt.scatter(X_tsne[idx, 0], X_tsne[idx, 1], label=label, alpha=0.7)
    plt.title("Trial-wise t-SNE Projection")
    plt.xlabel("Dim 1"); plt.ylabel("Dim 2")
    plt.legend(); plt.grid(True); plt.tight_layout()
    plt.show()


In [ ]:
from scipy.spatial.distance import cosine, euclidean

def compute_topographic_distances(topographies):
    """
    Compute pairwise distances between condition topographies.
    """
    labels = list(topographies.keys())
    dists = {}
    for i in range(len(labels)):
        for j in range(i+1, len(labels)):
            v1, v2 = topographies[labels[i]], topographies[labels[j]]
            dists[f"{labels[i]} vs {labels[j]}"] = {
                "cosine": cosine(v1, v2),
                "euclidean": euclidean(v1, v2)
            }
    return dists
